In [22]:
import torch
import torch.nn as nn

import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import ast

ModuleNotFoundError: No module named 'torch'

In [9]:
# Define Actor and Critic classes
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, action_dim)
    
    def forward(self, state):
        x = torch.relu(self.fc1(state))
        action_probs = torch.softmax(self.fc2(x), dim=-1)
        return action_probs

class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, state):
        x = torch.relu(self.fc1(state))
        value = self.fc2(x)
        return value

NameError: name 'nn' is not defined

In [ ]:
# Safe eval function
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []

# Preprocess the data
def preprocess_data(df):
    df = df.fillna(method='ffill')
    
    # Identify numeric features
    numeric_features = df.select_dtypes(include=[np.number])
    
    # Normalize numeric features if they exist
    if not numeric_features.empty:
        scaler = StandardScaler()
        df[numeric_features.columns] = scaler.fit_transform(numeric_features)
    
    # One-hot encode genres
    df['genre'] = df['genre'].apply(safe_literal_eval)
    df = df.explode('genre')
    df = pd.get_dummies(df, columns=['genre'])
    
    # Ensure 'emotions' column is evaluated into lists
    df['emotions'] = df['emotions'].apply(safe_literal_eval)
    
    return df

In [ ]:
# Load the configuration
with open('config.json', 'r') as f:
    config = json.load(f)

# Extract the values from the configuration
user_id = config["user_id"]
ego_centric_dim = config["ego_centric_dim"]
user_preferences_dim = config["user_preferences_dim"]
learning_rate = config["learning_rate"]
gamma = config["gamma"]
episodes = config["episodes"]
emotion_to_int = config["emotion_to_int"]
emotional_states = config["emotional_states"]

In [ ]:
# Load the datasets from the uploaded files
books_data = preprocess_data(pd.read_csv(r'D:\Com Acca\FYP\e18-4yp-Reinforcement-Learning-enabled-Recommendation-System-For-Emotional-Support\code\Recommendation System\Actor Critic Model\book_filtered_data.csv'))
movies_data = preprocess_data(pd.read_csv(r'D:\Com Acca\FYP\e18-4yp-Reinforcement-Learning-enabled-Recommendation-System-For-Emotional-Support\code\Recommendation System\Actor Critic Model\movie_filtered_data.csv'))
music_data = preprocess_data(pd.read_csv(r'D:\Com Acca\FYP\e18-4yp-Reinforcement-Learning-enabled-Recommendation-System-For-Emotional-Support\code\Recommendation System\Actor Critic Model\music_filtered_data.csv'))

In [ ]:
# Encode emotions in the datasets
def encode_emotions(df):
    df['emotions'] = df['emotions'].apply(lambda x: [emotion_to_int[emotion] for emotion in x])
    return df

books_data = encode_emotions(books_data)
music_data = encode_emotions(music_data)
movies_data = encode_emotions(movies_data)

In [ ]:
# Initialize state_dim and action_dim
state_dim = len(emotional_states) + ego_centric_dim + user_preferences_dim
action_dim = max(len(music_data), len(books_data), len(movies_data))

# Load the models
loaded_actor = Actor(state_dim, action_dim)
loaded_critic = Critic(state_dim)

loaded_actor.load_state_dict(torch.load('actor.pth'))
loaded_critic.load_state_dict(torch.load('critic.pth'))

loaded_actor.eval()
loaded_critic.eval()

In [ ]:
class RecommenderEnv:
    def __init__(self, user_id, music_data, books_data, movies_data, emotional_states):
        self.user_id = user_id
        self.music_data = music_data
        self.books_data = books_data
        self.movies_data = movies_data
        self.emotional_states = emotional_states
        self.state = None
        self.data = {
            'music': self.music_data,
            'books': self.books_data,
            'movies': self.movies_data
        }
        self.action_space = {
            'music': len(self.music_data),
            'books': len(self.books_data),
            'movies': len(self.movies_data)
        }

    def reset(self, specified_emotional_state=None):
        ego_centric_data = self.get_ego_centric_data()
        user_preferences = self.get_user_preferences(self.user_id)
        if specified_emotional_state is not None:
            emotional_state = specified_emotional_state
        else:
            emotional_state = np.zeros(len(self.emotional_states))
            emotional_state[np.random.choice(len(self.emotional_states))] = 1
        self.state = np.concatenate((emotional_state, ego_centric_data, user_preferences)).astype(np.float32)
        self.current_emotional_state = np.argmax(emotional_state)  # Store the current emotional state as an integer
        return self.state

    def step(self, action, item_type, user_feedback=None):
        action = action % self.action_space[item_type]  # Ensure the action is within the valid range
        next_state, reward, done = self._simulate_interaction(action, item_type, user_feedback)
        return next_state, reward, done

    def _simulate_interaction(self, action, item_type, user_feedback):
        next_state = self.reset()  # Reset to get the next state with new ego-centric data and user preferences
        
        if user_feedback is not None:
            reward = 1 if user_feedback == 'like' else -1
        else:
            reward = np.random.rand()
        
        done = np.random.rand() < 0.1
        return next_state, reward, done

    def get_recommendation(self, item_type):
        # Filter the activities based on the current emotional state
        valid_actions = self.data[item_type][self.data[item_type]['emotions'].apply(lambda x: self.current_emotional_state in x)].index
        if len(valid_actions) == 0:
            return None
        action = np.random.choice(valid_actions)
        return action

    @staticmethod
    def get_ego_centric_data():
        # This function should return a feature vector representing the user's environment
        # For simplicity, we return a random vector here
        return np.random.rand(5).astype(np.float32)

    @staticmethod
    def get_user_preferences(user_id):
        # This function should return a feature vector representing the user's preferences
        # For simplicity, we return a random vector here
        return np.random.rand(5).astype(np.float32)

In [ ]:
# Re-initialize the environment
loaded_env = RecommenderEnv(user_id=user_id, music_data=music_data, books_data=books_data, movies_data=movies_data, emotional_states=emotional_states)


In [ ]:
def get_user_emotional_state(emotional_states):
    print("Please enter your current emotional state:")
    for i, emotion in enumerate(emotional_states):
        print(f"{i}: {emotion}")

    while True:
        try:
            mood = int(input("Enter the number corresponding to your current emotional state: ").strip())
            if 0 <= mood < len(emotional_states):
                emotional_state = np.zeros(len(emotional_states))
                emotional_state[mood] = 1
                return emotional_state
            else:
                print(f"Invalid input. Please enter a number between 0 and {len(emotional_states) - 1}.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")


In [ ]:
def get_user_feedback():
    while True:
        feedback = input("Did you like the recommendation? (yes/no): ").strip().lower()
        if feedback in ['yes', 'no']:
            return 'like' if feedback == 'yes' else 'dislike'
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")


In [ ]:
def interactive_recommendation_system(env, actor, emotional_states):
    while True:
        user_emotional_state = get_user_emotional_state(emotional_states)
        state = env.reset(specified_emotional_state=user_emotional_state)
        state = torch.FloatTensor(state)

        while True:
            recommendations = {}
            feedbacks = {}

            for item_type in ['music', 'books', 'movies']:
                action = env.get_recommendation(item_type)
                if action is None:
                    print(f"No valid recommendations available for {item_type}.")
                    continue

                recommendation = env.data[item_type].iloc[action]
                recommendations[item_type] = recommendation
                print(f"Recommended {item_type[:-1]}: {recommendation['name']}")

                feedbacks[item_type] = get_user_feedback()

            for item_type, feedback in feedbacks.items():
                if item_type in recommendations:
                    next_state, reward, done = env.step(recommendations[item_type].name, item_type, feedback)
                    next_state = torch.FloatTensor(next_state)
                    state = next_state

            another_recommendation = input("Would you like another set of recommendations based on the same emotion? (yes/no): ").strip().lower()
            if another_recommendation != 'yes':
                break
        
        another_emotion = input("Would you like to provide a new emotional state for new recommendations? (yes/no): ").strip().lower()
        if another_emotion != 'yes':
            break



In [ ]:
# Example usage with the loaded models and environment
interactive_recommendation_system(loaded_env, loaded_actor, emotional_states)